In [1]:
#Import Python Libraries
#------------------------------
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
# from itertools import combinations
# import matplotlib.pyplot as plt
import csv
import networkx as nx
from networkx.algorithms import bipartite

In [2]:
#Read user_itemset_training.csv and Analyze
#------------------------------

data_u_is_train = pd.read_csv('./dataset/itemset_item_training.csv', sep=',', header=None)
# data_u_is_train[0] = 'IS'+data_u_is_train[0].astype(str)
# data_u_is_train[1] = 'I'+data_u_is_train[1].astype(str)
data_u_is_train[0] = data_u_is_train[0].astype(str)
data_u_is_train[1] = data_u_is_train[1].astype(str)
print(data_u_is_train.shape)
data_u_is_train.head()

(85822, 2)


,0,1
0,8571,18506
1,8571,33644
2,8571,32627
3,5725,14855
4,5725,28037


In [3]:
# checking jaccard_similarity -1
def jaccard_similarity(G, u, v):
    unbrs = set(G[u])
    vnbrs = set(G[v])
    return float(len(unbrs & vnbrs)) / len(unbrs | vnbrs)

In [4]:
#user projected graph
data_u_is_train_set = {(x[0], x[1]) for x in zip(data_u_is_train[0], data_u_is_train[1])}


X = data_u_is_train[1].unique()
Y = data_u_is_train[0].unique()


B = nx.Graph()
B.add_nodes_from(X, bipartite=0)
B.add_nodes_from(Y, bipartite=1)
B.add_edges_from(data_u_is_train_set)



G1 = bipartite.generic_weighted_projected_graph(B, X, weight_function= jaccard_similarity)

In [5]:
weight = nx.get_edge_attributes(G1, 'weight')

{('18506', '10866'): 0.058823529411764705,
 ('18506', '22364'): 0.07142857142857142,
 ('18506', '13424'): 0.1111111111111111,
 ('18506', '18485'): 0.04,
 ('18506', '19433'): 0.1,
 ('18506', '24869'): 0.06666666666666667,
 ('18506', '21015'): 0.1,
 ('18506', '11627'): 0.1,
 ('18506', '27178'): 0.09090909090909091,
 ('18506', '20896'): 0.1,
 ('18506', '5398'): 0.09090909090909091,
 ('18506', '20847'): 0.09090909090909091,
 ('18506', '15751'): 0.09090909090909091,
 ('18506', '22451'): 0.07692307692307693,
 ('18506', '16422'): 0.1111111111111111,
 ('18506', '1940'): 0.09090909090909091,
 ('18506', '27079'): 0.02564102564102564,
 ('18506', '9631'): 0.08333333333333333,
 ('18506', '32627'): 0.14285714285714285,
 ('18506', '18313'): 0.08333333333333333,
 ('18506', '16342'): 0.1,
 ('18506', '16180'): 0.07142857142857142,
 ('18506', '15690'): 0.09090909090909091,
 ('18506', '1665'): 0.09090909090909091,
 ('18506', '56'): 0.1,
 ('18506', '4921'): 0.1,
 ('18506', '11360'): 0.1,
 ('18506', '13777'

In [6]:
# CSV 파일 저장
def save_dict_to_csv(dictionary, file_path):
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['key1', 'key2', 'value'])  # 헤더 작성

        for key, value in dictionary.items():
            key1, key2 = key  # 튜플의 첫 번째와 두 번째 원소를 분리
            writer.writerow([key1, key2, value])
            
save_dict_to_csv(weight, 'dictionary_task2_weighted_item.csv')

In [7]:
# CSV 파일을 배열로 로드하는 함수
def load_csv_as_array(csv_file):
    data = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # 헤더(열 이름)를 건너뜁니다.
        for row in reader:
            data.append([int(row[0]), int(row[1]), float(row[2])])
    return data

def calculate_and_add_value(lst):
    grouped_data = {}
    
    # 첫 번째 요소 값을 기준으로 그룹화합니다.
    for sublist in lst:
        key = sublist[0]
        if key not in grouped_data:
            grouped_data[key] = []
        grouped_data[key].append(sublist[2])
    
    # 그룹화된 데이터를 기반으로 네 번째 요소를 추가합니다.
    for sublist in lst:
        key = sublist[0]
        if key in grouped_data:
            sum_value = sum(grouped_data[key])
            sublist.append(sublist[2]/sum_value*1000)
        else:
            sublist.append(0)  # 그룹화된 데이터가 없을 경우 기본값을 설정합니다.
    
    return lst


In [8]:
# CSV 파일 경로
csv_file = 'dictionary_task2_weighted_item.csv'

# CSV 파일을 배열로 로드
data = load_csv_as_array(csv_file)

# key1과 key2의 자리를 바꾼 배열 생성
swapped_data = [[row[1], row[0], row[2]] for row in data]

# 배열 연결 (concatenate)
concatenated_data = data + swapped_data

# key1의 값들을 기준으로 정렬된 배열 생성
temp_sorted_data = sorted(concatenated_data, key=lambda x: int(x[0]))

# True : weight = probability, False : weight = Jaccard Sim.
probability = True

if probability:
    sorted_data =calculate_and_add_value(temp_sorted_data)
else:
    sorted_data = temp_sorted_data


In [9]:
def binary_search(weight, sorted_array, target_keys):
    results = []  # 결과를 저장할 리스트

    for target_key in target_keys:
        left = 0
        right = len(sorted_array) - 1

        while left <= right:
            mid = (left + right) // 2
            current_key = sorted_array[mid][0]

            if current_key == target_key:
                key2 = sorted_array[mid][1]
                value = sorted_array[mid][2]
                results.append([key2, value * weight])  # 결과를 리스트에 추가

                # key1이 같은 다른 행들도 추가로 검색하여 결과에 추가
                i = mid - 1
                while i >= 0 and sorted_array[i][0] == target_key:
                    key2 = sorted_array[i][1]
                    value = sorted_array[i][2]
                    results.append([key2, value * weight])
                    i -= 1

                i = mid + 1
                while i < len(sorted_array) and sorted_array[i][0] == target_key:
                    key2 = sorted_array[i][1]
                    value = sorted_array[i][2]
                    results.append([key2, value * weight])
                    i += 1

                break

            if current_key < target_key:
                left = mid + 1
            else:
                right = mid - 1

    return results


# # Test
# sorted_array = [(28322, 40541, 0.125, 1.5), (28322, 706, 0.013, 0.8), (14958, 40541, 0.02, 1.2), (40541, 42524, 0.14, 0.9)]
# target_keys = [28322, 40541]

# related_keys = binary_search(1.0, sorted_array, target_keys)
# if len(related_keys) > 0:
#     print(related_keys)
# else:
#     print("Key1 not found")


In [10]:

def rank12(items):
    related_keys = sorted(binary_search(1.0, sorted_data,items), key=lambda x: x[-1], reverse=True)

    related_keys12 = related_keys[:100]

    for rank1 in related_keys:    
        temp = binary_search(rank1[-1], sorted_data, [rank1[0]])
        
        if len(related_keys12) == 100 and temp[-1][-1] > related_keys12[-1][-1]:
            related_keys12[-1] = temp[-1]
        
        related_keys12 += temp
        related_keys12 = sorted(related_keys12, key=lambda x: x[-1], reverse=True)[:100]
    
    return related_keys12
    
# items = [29213,23345]
# top100 = rank12(items)
    
# print(top100)
# print(len(top100))

In [11]:
filename = './dataset/itemset_item_test_query.csv'  # 불러올 CSV 파일명

valid_quary_dict = {}  # 변환된 딕셔너리를 저장할 변수

# CSV 파일 불러오기
with open(filename, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')

    for row in reader:
        key = int(row[0])
        value = int(row[1])

        if key in valid_quary_dict:
            valid_quary_dict[key].append(value)
        else:
            valid_quary_dict[key] = [value]

valid_quary_key = list(valid_quary_dict.keys())
valid_quary_value = list(valid_quary_dict.values())


In [12]:
my_dict = {}

itemset_num = len(valid_quary_dict)

top100_items_list = []
for i in range(itemset_num):
    key_i = valid_quary_key[i]
    items = valid_quary_dict[valid_quary_key[i]]
    top100 = rank12(items)
    top100_items = [sublist[0] for sublist in top100]
    top100_items_list.append(top100_items)
    my_dict[key_i] = top100_items


In [13]:
# key와 각 key마다 길이가 100인 리스트 형태의 value를 생성
values = [[i for i in range(100)] for _ in range(itemset_num)]


# CSV 파일 경로
csv_file = 'itemset_item_test_prediction.csv'

# CSV 파일에 데이터 쓰기
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['key'] + ['value_{}'.format(i) for i in range(100)])
    # writer.writeheader()
        
    # 각 key와 value를 CSV 파일에 쓰기
    for key, value in my_dict.items():
        row = {'key': key}
        row.update({'value_{}'.format(i): val for i, val in enumerate(value)})
        writer.writerow(row)


In [ ]:
# # answer csv 확인

# valid_answer = []  # 행을 저장할 리스트

# with open('./dataset/itemset_item_valid_answer.csv', 'r') as file:
#     reader = csv.reader(file)
#     for row in reader:
#         row = [int(element) for element in row]  # 각 요소를 int로 변환
#         valid_answer.append(row)
# # print(valid_answer)

# correct = 0
# for i in range(itemset_num):
#     missing_item = valid_answer[1]
#     if missing_item in top100_items_list[i]:
#         correct += 1
        
# accuracy = correct / itemset_num * 100
# print(accuracy)